## Set up the environment

In [1]:
import sagemaker as sage
from sagemaker import get_execution_role

role = get_execution_role()
sagemaker_session = sage.Session()
model_package_arn='arn:aws:sagemaker:[region]:[aws account id]:model-package/churn-mitigation'
s3_base_path='s3://[your s3 path]'

## Realtime Inference

In [2]:
from sagemaker import ModelPackage
model_name='churn-mitigation'
model = ModelPackage(name=model_name,
                     role=role,
                     model_package_arn=model_package_arn,
                     sagemaker_session=sagemaker_session)

In [3]:
endpoint_name='churn-mitigation-edp'

In [4]:
predictor = model.deploy(1, 'ml.p2.xlarge', endpoint_name=endpoint_name)

In [5]:
# connect to the endpoint
predictor = sage.predictor.RealTimePredictor(
    endpoint_name,
    sagemaker_session=sagemaker_session
)

In [6]:
# open s3 file
import io
import boto3
res = ""
s3_client = boto3.client('s3')
s3_response_object = s3_client.get_object(Bucket='Bucket-Name', Key='fileName')
object_content = s3_response_object['Body'].read()

res = predictor.predict(io.BytesIO(object_content))

In [7]:
# Delete endpoint
predictor.delete_endpoint()

## Batch Transform Job

In [8]:
s3_output_path=s3_base_path + '/output/'
transformer = sage.transformer.Transformer(
    base_transform_job_name='churn-mitigation-batch-transform',
    model_name=model_name,
    instance_count=1,
    instance_type='ml.m5.large',
    output_path=s3_output_path,
    max_payload=100,
    sagemaker_session=sagemaker_session
)

In [9]:
transformer.transform(
    s3_base_path + '/inputData/'
)

# wait until prediction job is completed
transformer.wait()


#### Inspect the Batch Transform Output in S3

In [10]:
s3_client = sage.s3.S3Downloader
s3_client.read_file(s3_base_path + '/output/input.json.out')